In [1]:
import numpy as np
import qiskit
import tqix
import sys, math
sys.path.insert(1, '../')
import qtm.constant, qtm.base

In [ ]:

# Import packages.
import cvxpy as cp
import numpy as np

def fit(state):
    n = 4
    trial_state = cp.Variable((n, 1)) # reconstructed state
    # Constraint for density matrix
    # https://quantumcomputing.stackexchange.com/questions/2336/how-to-check-if-a-matrix-is-a-valid-density-matrix
    constraints = [trial_state @ trial_state.T >> 0] # density matrix is semi-define
    # constraints = [cp.trace(trial_state @ trial_state.T) == 1 ]
    # Trace between state and reconstructed state
    prob = cp.Problem(cp.Minimize(1 - calculate_TML(trial_state.value)),
                  constraints)
    prob.solve()
    # Print result.
    print("The optimal value is", prob.value)
    print("A solution X is")
    print(trial_state.value)

In [9]:
def calculate_TML(trial_state):
    ###################
    ## Calculate Pr
    ###################
    if (trial_state) is None:
        return -100
    model = tqix.qmeas(trial_state, 'MUB')
    pr = model.probability()
    for i in range(0, len(pr)):
        if pr[i] == 0:
            pr[i] = 10**(-10)
    ###################
    ## Calculate fr
    ###################
    model = tqix.qmeas(state, 'MUB')
    cdf_model = tqix.qsim(model, niter = qtm.constant.num_shots, backend='cdf')
    fr = np.real(cdf_model.probability())
    ###################
    ## Calculate Tml
    ###################
    tml = 0
    for i in range(0, len(pr)):
        tml += np.real(fr[i]*np.log(pr[i]))
    return tml

In [13]:
num_qubits = 2

# psi = 2*np.random.rand(2**num_qubits)-1
# psi = psi / np.linalg.norm(psi)
# qc = qiskit.QuantumCircuit(num_qubits, num_qubits)
# qc.initialize(psi, range(0, num_qubits))
# state = qiskit.quantum_info.Statevector(qc).data
# state = np.real(state.reshape((2**num_qubits, 1)))
state = tqix.random(4)
max_tml = 0
tmls = []
for i in range(0, 10000):
    # trial_state = np.random.uniform(low = 0, high = 2*np.pi, size = (2**num_qubits, 1))
    # trial_state = trial_state / np.linalg.norm(trial_state)
    trial_state = tqix.random(4)
    tml = calculate_TML(trial_state)
    tmls.append(tml)
    if i == 0 or max_tml < tml:
        max_tml = tml
        reconstructed_state = trial_state

        # if fidelity > 0.99:
        #     print('Founded')
        #     break
        if tml >= 1:
            break
# state = np.squeeze(state, axis = 1)
# reconstructed_state = np.squeeze(reconstructed_state, axis = 1)
# trace, fidelity = qtm.base.get_metrics(state, reconstructed_state)
# print(tml)
# print("State: ", state)
# print("Reconstructed state: ", reconstructed_state)
# print(fidelity)
print(max(tmls))
print(min(tmls))


-5.638341259240184
-15.668346137072346


first, we change ghz(1) by Haar random called from Qiskit. let fix N = 2.
then we calculate the cdf_model probabilites so, now we have f_r.

len(f_r) = (2^n + 1)*2^n
f_r(1) = (2^1 + 1)*2
Next, we calculate the trial probabilities, to calculate the trial probabilities, you can call the probability from model (not cdf_model)

1. We have $f_r$ based on measure random unknown state
2. For each $|\psi_{trial}\rangle$, we have different $P_r$.
3. And $T_{ML}(\psi_{trial})=\Sigma (f_r.\log(P_R)) \in C$?

We repeat 2-3 until $T_{ML}(\psi_{trial})$ max and $\psi_{trial}$ is $|\mu\rangle$

<img width = 500px src = '../../images/classical_tomography.jpg'/>